Problem Statement.

Given an undirected tree, return its diameter: the number of edges in a longest path in that tree.

The tree is given as an array of edges where edges[i] = [u, v] is a bidirectional edge between nodes u and v.  Each node has labels in the set {0, 1, ..., edges.length}.

 

Example 1:

Input: edges = [[0,1],[0,2]]
Output: 2
Explanation: 
A longest path of the tree is the path 1 - 0 - 2.

Example 2:

Input: edges = [[0,1],[1,2],[2,3],[1,4],[4,5]]
Output: 4
Explanation: 
A longest path of the tree is the path 3 - 2 - 1 - 4 - 5.

 

Constraints:

    0 <= edges.length < 10^4
    edges[i][0] != edges[i][1]
    0 <= edges[i][j] <= edges.length
    The given edges form an undirected tree.

# Time Limit Exceeded - DFS - O(N ^ 2) runtime, O(N) space

In [5]:
from typing import List
from collections import defaultdict
from heapq import heappush, heappop

class Solution:
    def treeDiameter(self, edges: List[List[int]]) -> int:
        self.graph = defaultdict(list)
        
        for a, b in edges:
            self.graph[a].append(b)
            self.graph[b].append(a)
            
        maxPath = 0
        
        for source in self.graph:
            minheap = []
            for nextNode in self.graph[source]:
                curVal = 1 + self.findPathLen(nextNode, {source, nextNode})
                heappush(minheap, curVal)
                if len(minheap) > 2: heappop(minheap)
            maxPath = max(maxPath, sum(minheap))
            
        return maxPath
    
    def findPathLen(self, source, visited) -> int:
        maxVal = 0
        for nextNode in self.graph[source]:
            if nextNode not in visited:
                curVal = 1 + self.findPathLen(nextNode, visited.union({nextNode}))
                maxVal = max(maxVal, curVal)
      
        return maxVal

# DFS - O(N) runtime, O(N) space

In [13]:
from typing import List
from collections import defaultdict

class Solution:
    def treeDiameter(self, edges: List[List[int]]) -> int:
        # build the adjacency list representation of the graph.
        graph = defaultdict(set)
        for u, v in edges:
            graph[u].add(v)
            graph[v].add(u)

        diameter = 0

        def dfs(curr, visited):
            """
                return the max distance
                  starting from the 'curr' node to its leaf nodes
            """
            nonlocal diameter

            # the top 2 distance starting from this node
            top_1_distance, top_2_distance = 0, 0

            distance = 0
            visited.add(curr)

            for neighbor in graph[curr]:
                if neighbor not in visited:
                    distance = 1 + dfs(neighbor, visited)

                if distance > top_1_distance:
                    top_1_distance, top_2_distance = distance, top_1_distance
                elif distance > top_2_distance:
                    top_2_distance = distance

            # with the top 2 distance, we can update the current diameter
            diameter = max(diameter, top_1_distance + top_2_distance)

            return top_1_distance

        visited = set()
        dfs(0, visited)

        return diameter

# Farthest Distance BFS - O(N) runtime, O(N) space

In [9]:
from typing import List
from collections import deque, defaultdict

class Solution:
    def treeDiameter(self, edges: List[List[int]]) -> int:
        # build the adjacency list representation of the graph
        graph = defaultdict(set)
        for u, v in edges:
            graph[u].add(v)
            graph[v].add(u)

        def bfs(start):
            """
             return the farthest node from the 'start' node
               and the distance between them.
            """
            
            queue = deque([start])
            visited = {start}
            distance = -1
            last_node = None
            
            while queue:
                n = len(queue)
                for _ in range(n):
                    next_node = queue.popleft()
                    for neighbor in graph[next_node]:
                        if neighbor not in visited:
                            visited.add(neighbor)
                            queue.append(neighbor)
                            last_node = neighbor
                distance += 1

            return last_node, distance

        # 1). find one of the farthest nodes
        farthest_node, distance_1 = bfs(0)
        # 2). find the other farthest node
        #  and the distance between two farthest nodes
        another_farthest_node, distance_2 = bfs(farthest_node)

        return distance_2

# Centroid BFS - O(N) runtime, O(N) space

In [11]:
from typing import List
from collections import defaultdict

class Solution:
    def treeDiameter(self, edges: List[List[int]]) -> int:
        # build the adjacency list representation of the graph.
        graph = defaultdict(set)
        for u, v in edges:
            graph[u].add(v)
            graph[v].add(u)

        # find the outer most nodes, _i.e._ leaf nodes
        leaves = []
        for vertex, links in graph.items():
            if len(links) == 1:
                leaves.append(vertex)

        # "peel" the graph layer by layer,
        #   until we have the centroids left.
        layers = 0
        vertex_left = len(edges) + 1
        while vertex_left > 2:
            vertex_left -= len(leaves)
            next_leaves = []
            for leaf in leaves:
                # the only neighbor left on the leaf node.
                neighbor = graph[leaf].pop()
                graph[neighbor].remove(leaf)
                if len(graph[neighbor]) == 1:
                    next_leaves.append(neighbor)
            layers += 1
            leaves = next_leaves

        return layers * 2 + (0 if vertex_left == 1 else 1)

In [12]:
instance = Solution()
instance.treeDiameter([[0,1],[1,2],[2,3],[1,4],[4,5]])

4